# Algorithme MCMC

Une fois obtenu le MLE, nous utilisons un un algorithme de Monte-Carlo pour affiner le résultat : 
1) On pose $Z_0 = Z_{MLE}$ et $\alpha_0 \sim Exp(\lambda)$
2) On tire $\overset{-}{Z}_{k+1} \sim \mathcal{N}(Z_k, \sigma^2 I_{n.k})$ ($n$ taille de Z, $k$ dimension des points)
3) On réalise la transformation procustéenne : $\overset{\sim}{Z} = \underset{T}{argmin}\ tr \left[\left(
Z_0-T \overset{-}{Z}_{k+1}\right)^t \left( Z_0-T \overset{-}{Z}_{k+1}\right) \right] $ (T est une opération combinant la translation, la rotation et la symétrie).
4) On accepte $\overset{\sim}{Z}_{k+1}$ en tant  que $Z_{k+1}$ avec probilité $max \left\{1, \frac{\mathbb{P}(Y|\overset{\sim}{Z}_{k+1}, \alpha)}{\mathbb{P}(Y|Z_k, \alpha)} \frac{\pi_1(\overset{\sim}{Z})}{\pi_1(Z_k)} \right\}$, sinon on pose $Z_{k+1}=Z_k$.
5) On tire $\overset{\sim}{\alpha} \sim Exp(\lambda)$ (on ne peut tirer selon une loi normale centrée en $\alpha$ au risque d'avoir un $\alpha$ négatif.)
6) On accepte $\overset{\sim}{\alpha}$ en tant  que $\alpha_{k+1}$ avec probilité $max \left\{1, \frac{\mathbb{P}(Y|Z_{k+1}, \overset{\sim}{\alpha)}}{\mathbb{P}(Y|Z_{k+1}, \alpha)} \frac{\pi_2(\overset{\sim}{\alpha})}{\pi_2(\alpha_k)} \right\}$, sinon on pose $\alpha_{k+1}=\alpha_k$

_L'étape 3 (la transformation procustéenne) est nécessaire car notre modèle ne tient compte que des distances. Donc pour un nuage de point donnée, tout nuage de point obtenu par translation, rotation ou symétrie de ce nuage de point donne les mêmes résultats. Comme on veut s'assurer de la bonne convergence du modèle, on force les nouveaux points obtenus à être le plus proche du nuage de point initial via ces transformations. Cependant, on ne fait pas de mise à l'échelle contrairement à ce qui peut se faire dans d'autres transformations procustéenne, car on modifierait alors la distance._

On prendra pour loi $\pi_1$ une loi normale centrée, de variance suffisament grande, et pour loi $\pi_2$ une loi normale exponentielle, de paramètre 2, ce paramètre permettant de ne pas d'avoir d'$\alpha$ trop petit ou trop grand.

On importe les bibliothèques :

In [2]:
!py -m pip install numpy
!py -m pip install scipy
import numpy as np
#from scipy.stats import norm
from scipy.linalg import orthogonal_procrustes
from MLE import MLE


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


On définit les lois $\pi_1$ et $\pi_2$ :

In [ ]:
def pi_1(Z, sigma=10):
    '''
    log de la loi à priori des Z (à une constante près)
    '''
    return np.sum(np.linalg.norm(Z)**2 / (2 * sigma**2))

def pi_2(alpha, lamb=2):
    '''
    log de la loi à priori des alpha (à une constante près)
    '''
    return max(- lamb*alpha, 0)

On définie le log de la probabilité d'acceptation :

In [ ]:
def test_Z(alpha, Z_tilde, Z, Y, sigma=10):
    """
    renvoie le log de probabilité d'acceptation pour Z
    """
    test = 0
    for i in range(len(Z)):
        for j in range(len(Z)):
            if i !=j:
                eta = alpha - np.linalg.norm(Z[i]-Z[j])
                eta_tilde = alpha - np.linalg.norm(Z_tilde[i]-Z_tilde[j])
                test += Y[i][j]*eta_tilde-np.log(1+np.exp(eta_tilde))-Y[i][j]*eta-np.log(1+np.exp(eta))
        test += pi_1(Z_tilde, sigma)-pi_1(Z, sigma)
    return test

def test_alpha(alpha_tilde, alpha, Z, Y, lamb=2):
    """
    renvoie le log de probabilité d'acceptation pour alpha
    """
    test = 0
    for i in range(len(Z)):
        for j in range(len(Z)):
                if i != j:
                    eta = alpha - np.linalg.norm(Z[i]-Z[j])
                    eta_tilde = alpha_tilde - np.linalg.norm(Z[i]-Z[j])
                    test += Y[i][j]*eta_tilde - np.log(1+np.exp(eta_tilde)) - Y[i][j]*eta - np.log(1+np.exp(eta))
    test += pi_2(alpha_tilde) - pi_2(alpha)
    return test

On définie une fonction réalisant une étape de l'algorithme de Monte Carlo :

In [ ]:
def MCMC(alpha, Z, Y, pas1=1, pas2=0.5, lamb=2, sigma=10):
    """
    Cette fonction prend un couple (alpha,Z), et renvoie un nouveau couple (alpha,Z).
    Le nouveau Z est tiré selon une loi normale centrée en Z, de variance pas1**2. Puis accepté, avec pour prior
    la loi normale centrée, de variance sigma**2.
    Le nouveau alpha est tiré selon une loi normale centrée en Z, de variance pas2**2. Puis accepté, avec pour prior
    la loi exponentielle de paramètre lambda.
    """
    Z_tilde = Z + np.random.normal(0, pas1, size=(Z.shape[0], Z.shape[1]))
    Z_tilde = Z_tilde - np.mean(Z_tilde, axis=0)
    T, d = orthogonal_procrustes(Z, Z_tilde)
    Z_tilde = np.dot(Z_tilde, T)
    test = test_Z(alpha, Z_tilde, Z, Y, sigma)
    if np.random.random() < np.exp(test) :
        Z = Z_tilde
    alpha_tilde = alpha + np.random.normal(0, pas2)
    test = test_alpha(alpha_tilde, alpha, Z, Y, lamb)
    if np.random.random() < np.exp(test) : 
        alpha = alpha_tilde
    return alpha, Z